In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION
from lsst.obs.lsst import LsstCam
from lsst.geom import SpherePoint,Angle,Extent2I,Box2I,Extent2D,Point2D, Point2I
from lsst.summit.utils.efdUtils import makeEfdClient
from lsst.summit.utils.simonyi.mountAnalysis import calculateMountErrors
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId

In [ ]:
butler = Butler('LSSTCam', collections=["LSSTCam/raw/all", "LSSTCam/calib", "LSSTCam/runs/quickLook"])
instrument = 'LSSTCam'
camera = LsstCam.getCamera()
client = makeEfdClient()

In [ ]:
def DeltaAltAz(ra, dec, pressure, hum, temperature, wl, time1, time2):
    # This calculates the change in AltAz during an exposure
    # given the RA/Dec and other variables
    skyLocation = SkyCoord(ra*u.deg, dec*u.deg)
    altAz1 = AltAz(obstime=time1, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    altAz2 = AltAz(obstime=time2, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    obsAltAz1 = skyLocation.transform_to(altAz1)
    obsAltAz2 = skyLocation.transform_to(altAz2)
    # 1 is at the beginning of the exposure, 2 is at the end
    # These are all in degrees
    el1 = obsAltAz1.alt.deg
    az1 = obsAltAz1.az.deg
    el2 = obsAltAz2.alt.deg
    az2 = obsAltAz2.az.deg
    print(f"AzStart={az1:.6f}, ElStart={el1:.6f}, AzEnd={az2:.6f}, ElEnd={el2:.6f}")
    # Change values are the change from the beginning to the end of the exposure, in arcseconds
    azChange = (az2 - az1) * 3600.0
    elChange = (el2 - el1) * 3600.0
    print(f"AzChange={azChange:.2f}, elChange={elChange:.2f}")
    return [azChange, elChange]

In [ ]:
# Below is the wavelength center point for the LSST filters:
wavelengths = {'u':3671, 'g':4827, 'r':6223, 'i':7546, 'z':8691, 'y':9712}
detector = camera['R22_S11']
bbox = detector.getBBox()

def CalculateDrift(expId):
    rawExp = butler.get('raw', detector=94, exposure=expId, instrument=instrument)
    md = rawExp.getMetadata()
    filter = md['FILTBAND']
    wl = wavelengths[filter] * u.angstrom
    pressure = md['PRESSURE'] * u.pascal
    temperature = md['AIRTEMP'] * u.Celsius
    hum = md['HUMIDITY'] / 100.0
    time1 = Time(md['MJD-BEG'], format='mjd', scale='tai')
    time2 = Time(md['MJD-END'], format='mjd', scale='tai')
    raPoint = md['RA']
    decPoint = md['DEC']
    el = md['ELSTART']
    print(f"We think telescope is pointed at (RA, Dec), ({raPoint:.6f}, {decPoint:.6f})")
    [azChangePoint, elChangePoint] = DeltaAltAz(raPoint, decPoint, pressure, hum, temperature, wl, time1, time2)
    calExp = butler.get('preliminary_visit_image', detector=94, visit=expId, instrument=instrument)
    cWcs = calExp.getWcs()
    if not cWcs:
        return None
    calExpSkyCenter = cWcs.pixelToSky(Point2D(bbox.centerX, bbox.centerY))
    raReal = calExpSkyCenter.getRa().asDegrees()
    decReal = calExpSkyCenter.getDec().asDegrees()
    deltaRa = (raReal - raPoint) * 3600.0
    deltaDec = (decReal - decPoint) * 3600.0
    print(f"Telescope is actually pointed at (RA, Dec), ({raReal:.6f}, {decReal:.6f})")
    print(f" Pointing error in RA, Dec is ({deltaRa:.1f}, {deltaDec:.1f}) arcseconds")
    [azChangeReal, elChangeReal] = DeltaAltAz (raReal, decReal, pressure, hum, temperature, wl, time1, time2)
    azDrift = azChangeReal - azChangePoint
    elDrift = elChangeReal - elChangePoint
    totalDrift = np.sqrt(elDrift**2 + (azDrift * np.cos(el * np.pi / 180.0))**2)
    print(f" For {expId}, Azimuth drift = {azDrift:.2f} arcseconds, Elevation drift = {elDrift:.2f} arcseconds, Total drift = {totalDrift:.2f} arcseconds.")
    return [azDrift, elDrift]

In [ ]:
dayObs = 20251023
table = pd.read_json(f'/project/rubintv/LSSTCam/sidecar_metadata/dayObs_{dayObs}.json').T
table = table.sort_index()
expIds = []
azDrifts = []
elDrifts = []
for i in range(1, len(table)+1):
    expId = int(dayObs * 1.0E5 + i)
    try:
        [azDrift, elDrift] = CalculateDrift(expId)
        azDrifts.append(azDrift)
        elDrifts.append(elDrift)
        expIds.append(expId)
    except:
        continue
    if expId % 10 == 0:
        print(f"Finished {expId}")

In [ ]:
expIds = np.array(expIds)
seqNums = expIds - dayObs * 1E5
plt.title(f"Image drift due to pointing model error {dayObs}")
plt.scatter(seqNums, azDrifts, label='Az')
plt.scatter(seqNums, elDrifts, label='El')
plt.ylim(-1.0, 1.0)
plt.xlabel("Sequence number")
plt.ylabel("Drift (arcseconds)")
plt.legend()
plt.savefig(f"/home/cslage/DATA/Pointing_Model_Drift_{dayObs}")

In [ ]:
expId = 2025102600506
[azDrift, elDrift] = CalculateDrift(expId)
print(f"For {expId}, Az drift is {azDrift:.6f}, El drift is {elDrift:.6f} arcseconds")

AzStart=89.775275, ElStart=60.376492, AzEnd=89.709292, ElEnd=60.488141

In [ ]:
instrument = 'LSSTCam'
expId = 2025102600506
detector =94
instrument = 'LSSTCam'
raw = butler.get('raw', detector=detector, exposure=expId, instrument=instrument)
mData = raw.getMetadata()
azChange = (mData['AZEND'] - mData['AZSTART']) * 3600.0
elChange = (mData['ELEND'] - mData['ELSTART']) * 3600.0
print(f"AzStart={mData['AZSTART']:.6f}, ElStart={mData['ELSTART']:.6f}, AzEnd={mData['AZEND']:.6f}, ElEnd={mData['ELEND']:.6f}")
print(f"AzChange={azChange:.2f}, elChange={elChange:.2f}")

In [ ]:
expId = 2025102600479
dataId = {'exposure':expId, 'instrument':instrument}
expRecord = getExpRecordFromDataId(butler, dataId)
(mountErrors, mountData) = calculateMountErrors(expRecord, client)
efd_azs = mountData.azimuthData['actualPosition'].values
point_az = (efd_azs[0] + efd_azs[-1]) / 2.0
efd_els = mountData.elevationData['actualPosition'].values
point_el = (efd_els[0] + efd_els[-1]) / 2.0
efd_rot = mountData.rotationData['actualPosition'].values
rot = (efd_rot[0] + efd_rot[-1]) / 2.0
azStart = efd_azs[0]
azEnd = efd_azs[-1]
elStart = efd_els[0]
elEnd = efd_els[-1]
azChange = (azEnd - azStart) * 3600.0
elChange = (elEnd - elStart) * 3600.0
print(f"AzStart={azStart:.6f}, ElStart={elStart:.6f}, AzEnd={azEnd:.6f}, ElEnd={elEnd:.6f}")
print(f"AzChange={azChange:.2f}, elChange={elChange:.2f}")
azDelta = (azStart - mData['AZSTART']) * 3600.0
elDelta = (elStart - mData['ELSTART']) * 3600.0
print(f"Deltas between EFD and headers: Az={azDelta:.2f}, El={elDelta:.2f} arseconds")

In [ ]:
efd_els_478 = efd_els
times = mountData.azimuthData.actualPositionTimestamp.values
times -= times[0]
times_478 = times
(efd_els_478[-1] - efd_els_478[0]) * 3600.0

In [ ]:
efd_els_479 = efd_els
times = mountData.azimuthData.actualPositionTimestamp.values
times -= times[0]
times_479 = times
(efd_els_479[-1] - efd_els_479[0]) * 3600.0

In [ ]:
plt.plot(times_478, efd_els_478)
plt.plot(times_479, efd_els_479)

In [ ]:
mData['RASTART']

In [ ]:
mData['RAEND']

In [ ]:
mData['RA']

In [ ]:
AltAz?